In [39]:
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print('all imported')

all imported


In [40]:
def readImage(filePath):
    img = Image.open(filePath)
    
    size=(30,40)
    #resize image
    out = img.resize(size)

    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(out)
    numpydata = np.repeat(numpydata[:, :, np.newaxis], 3, axis=2)
    # <class 'numpy.ndarray'>
    #print(type(numpydata))

    #  shape
    #print(numpydata.shape)
    return numpydata
print(readImage('../../content/Benign/B_3141_1.RIGHT_CC.LJPEG.1_highpass.gif'))

[[[ 74  74  74]
  [ 83  83  83]
  [ 86  86  86]
  ...
  [108 108 108]
  [107 107 107]
  [102 102 102]]

 [[  8   8   8]
  [  8   8   8]
  [  7   7   7]
  ...
  [  9   9   9]
  [ 14  14  14]
  [ 25  25  25]]

 [[  8   8   8]
  [  7   7   7]
  [  7   7   7]
  ...
  [  9   9   9]
  [ 15  15  15]
  [ 27  27  27]]

 ...

 [[  8   8   8]
  [  7   7   7]
  [  7   7   7]
  ...
  [128 128 128]
  [ 87  87  87]
  [ 84  84  84]]

 [[  8   8   8]
  [  8   8   8]
  [  7   7   7]
  ...
  [ 66  66  66]
  [ 73  73  73]
  [ 77  77  77]]

 [[ 87  87  87]
  [ 70  70  70]
  [ 61  61  61]
  ...
  [ 85  85  85]
  [ 79  79  79]
  [ 73  73  73]]]


In [41]:
listFiles = os.listdir('../../content/Benign/')
print(listFiles[:5])

['B_3091_1.LEFT_CC.LJPEG.1_highpass.gif', 'B_3091_1.LEFT_MLO.LJPEG.1_highpass.gif', 'B_3093_1.LEFT_CC.LJPEG.1_highpass.gif', 'B_3093_1.LEFT_MLO.LJPEG.1_highpass.gif', 'B_3094_1.LEFT_CC.LJPEG.1_highpass.gif']


In [42]:
#load Benign
listCancer = []
listResults = []
for elem in listFiles:
    img = readImage('../../content/Benign/'+elem)
    listCancer += [img]
    listResults += [1]
print(listCancer[:3])
print(img.shape)

[array([[[  5,   5,   5],
        [ 21,  21,  21],
        [ 13,  13,  13],
        ...,
        [  7,   7,   7],
        [  5,   5,   5],
        [  2,   2,   2]],

       [[ 23,  23,  23],
        [ 14,  14,  14],
        [  9,   9,   9],
        ...,
        [  1,   1,   1],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 23,  23,  23],
        [ 14,  14,  14],
        [  9,   9,   9],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[ 29,  29,  29],
        [ 17,  17,  17],
        [ 10,  10,  10],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[128, 128, 128],
        [ 17,  17,  17],
        [ 12,  12,  12],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[128, 128, 128],
        [128, 128, 128],
        [ 14,  14,  14],
        ...,
        [  7,   7,   7],
        [  6,   6,   6],
        [  6,   6,   6]

In [43]:
listFiles = os.listdir('../../content/Normal/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('../../content/Normal/'+elem)
    listResults += [0]
    listCancer += [img]

['A_0002_1.LEFT_CC.LJPEG.1_highpass.gif', 'A_0002_1.LEFT_MLO.LJPEG.1_highpass.gif', 'A_0002_1.RIGHT_CC.LJPEG.1_highpass.gif', 'A_0002_1.RIGHT_MLO.LJPEG.1_highpass.gif', 'A_0003_1.LEFT_CC.LJPEG.1_highpass.gif']


In [44]:
listFiles = os.listdir('../../content/Malign/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('../../content/Malign/'+elem)
    listResults += [2]
    listCancer += [img]

['B_3006_1.LEFT_CC.LJPEG.1_highpass.gif', 'B_3006_1.LEFT_MLO.LJPEG.1_highpass.gif', 'B_3011_1.LEFT_CC.LJPEG.1_highpass.gif', 'B_3011_1.LEFT_MLO.LJPEG.1_highpass.gif', 'B_3011_1.RIGHT_CC.LJPEG.1_highpass.gif']


In [45]:
print(listResults)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [46]:
import numpy as np
lst = np.random.permutation(len(listResults))
lst = list(lst)

In [47]:
lst

[97,
 121,
 222,
 176,
 57,
 95,
 98,
 110,
 143,
 83,
 277,
 119,
 206,
 126,
 163,
 140,
 146,
 291,
 90,
 158,
 113,
 3,
 142,
 199,
 204,
 48,
 148,
 245,
 275,
 50,
 102,
 265,
 218,
 12,
 69,
 193,
 118,
 55,
 139,
 129,
 258,
 103,
 44,
 279,
 249,
 285,
 78,
 31,
 185,
 54,
 32,
 169,
 248,
 42,
 45,
 181,
 39,
 294,
 94,
 88,
 117,
 93,
 59,
 133,
 298,
 4,
 105,
 174,
 141,
 125,
 191,
 235,
 131,
 259,
 268,
 180,
 92,
 22,
 91,
 33,
 29,
 238,
 8,
 229,
 96,
 228,
 293,
 212,
 135,
 154,
 23,
 227,
 60,
 283,
 62,
 68,
 1,
 73,
 263,
 290,
 112,
 152,
 282,
 213,
 262,
 208,
 5,
 267,
 37,
 189,
 84,
 24,
 221,
 274,
 116,
 157,
 120,
 247,
 132,
 136,
 155,
 75,
 149,
 242,
 124,
 215,
 186,
 172,
 200,
 156,
 13,
 233,
 223,
 159,
 253,
 271,
 217,
 224,
 100,
 137,
 197,
 30,
 209,
 166,
 108,
 85,
 286,
 251,
 26,
 67,
 19,
 16,
 236,
 296,
 14,
 295,
 241,
 239,
 288,
 58,
 34,
 82,
 51,
 46,
 104,
 287,
 237,
 234,
 187,
 25,
 47,
 289,
 72,
 167,
 243,
 195,
 276,
 2

In [48]:
data = [listResults[i] for i in lst]

In [49]:
sz = (len(lst) * 80) // 100
train = [data[i] for i in range(sz)]
test = [data[i] for i in range(sz, len(lst))]
print(train)
print(test)

[1, 0, 2, 0, 1, 1, 1, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 0, 2, 2, 1, 1, 0, 0, 1, 0, 0, 2, 0, 1, 2, 2, 2, 1, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 0, 1, 1, 0, 2, 1, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 0, 0, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 0, 0, 2, 2, 2, 2, 1, 2, 1, 0, 1, 1, 2, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 2, 0, 1, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 1, 2, 0, 0, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 2, 2, 2, 0, 1, 1, 2, 1, 0, 2, 0, 2, 2, 2, 1, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 0, 1, 1, 1, 0, 0, 1, 1, 1, 2, 2, 0, 1, 2, 2, 0, 1, 0, 2, 1, 0, 1, 2, 0, 2, 0, 0, 2, 0, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 0, 2, 0, 2, 1, 0, 1, 2]
[1, 0, 0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 1, 0, 2, 1, 2, 0, 0, 1, 2, 0, 0, 1, 0, 2, 1, 1, 2, 1, 0, 0, 2, 0, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 0, 2, 2, 0, 1, 2, 1, 2, 2, 0, 2, 1, 0]


In [53]:
from statistics import mode
rsp = mode(train)
pred = []
for i in range(len(test)):
    pred+=[rsp]
acc = 0
for i in range(len(test)):
    if pred[i] == test[i]:
        acc+=1
acc = acc / len(test)
acc
precision = precision_score(test, pred, average='weighted')
recall = recall_score(test, pred, average='weighted')
print("acc: ", acc)
print("precision: ", precision)
print("recall: ", recall)

acc:  0.2833333333333333
precision:  0.08027777777777777
recall:  0.2833333333333333


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
